## Biblioteca FlashRank

In [4]:
from flashrank import Ranker, RerankRequest

In [20]:
query = "Quais são os principais desafios e aplicações da IA generativa na indústria atual?"
passages = [
   {
      "id": 1,
      "text": "A IA generativa tem ganhado destaque na indústria, especialmente com modelos como GPT e Diffusion, capazes de criar conteúdos originais e inovadores em diversas áreas. Sua aplicação vai desde geração de texto e imagens até novos designs e modelos de negócio.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 2,
      "text": "Na indústria atual, a IA generativa é utilizada em uma variedade de setores, incluindo chatbots para atendimento ao cliente, criação de imagens personalizadas para campanhas de marketing, design de produtos inovadores e produção de conteúdos audiovisuais, como roteiros e dublagens.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 3,
      "text": "Além das aplicações, a IA generativa enfrenta desafios importantes, como o viés nos modelos, a criação de deepfakes e a desinformação. Existe também a preocupação com o impacto nos empregos, especialmente em áreas de produção de conteúdo e design.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 4,
      "text": "A IA generativa pode ter um impacto significativo na redução de custos de produção, acelerando processos criativos e aumentando a eficiência. No entanto, o alto custo de treinamento de modelos avançados e a necessidade de grande poder computacional ainda representam barreiras para muitas empresas.",
      "meta": {"importância": "média"}
   },
   {
      "id": 5,
      "text": "Embora a IA generativa traga avanços significativos em personalização de conteúdo, como recomendações de produtos e serviços em plataformas de streaming, seu uso mais generalizado em marketing digital e publicidade também levanta questões sobre privacidade e ética no uso de dados pessoais.",
      "meta": {"importância": "média"}
   },
   {
      "id": 6,
      "text": "O aquecimento global é uma das questões mais urgentes da atualidade, com impactos profundos nos ecossistemas e nas sociedades. As emissões de gases do efeito estufa continuam a aumentar, o que gera mudanças climáticas e eventos extremos cada vez mais frequentes.",
      "meta": {"importância": "baixa"}
   },
   {
      "id": 7,
      "text": "A história da arte é fascinante, cobrindo diversos períodos e movimentos que refletem as transformações culturais e sociais ao longo do tempo. Do Renascimento ao Modernismo, as expressões artísticas têm sido um reflexo dos avanços e desafios da humanidade.",
      "meta": {"importância": "baixa"}
   },
   {
      "id": 8,
      "text": "A exploração espacial continua a ser uma fronteira importante para a ciência e a tecnologia. Missões como as de Marte e a construção de estações espaciais internacionais têm ampliado o conhecimento humano sobre o universo e possibilitado novas tecnologias aplicáveis na Terra.",
      "meta": {"importância": "baixa"}
   }
]


In [6]:
ranker = Ranker(max_length=128)
rerankrequest = RerankRequest(query=query, passages=passages)

In [7]:
results = ranker.rerank(rerankrequest)
results

[{'id': 1,
  'text': 'A IA generativa tem ganhado destaque na indústria, especialmente com modelos como GPT e Diffusion, capazes de criar conteúdos originais e inovadores em diversas áreas. Sua aplicação vai desde geração de texto e imagens até novos designs e modelos de negócio.',
  'meta': {'importância': 'alta'},
  'score': 0.97679156},
 {'id': 3,
  'text': 'Além das aplicações, a IA generativa enfrenta desafios importantes, como o viés nos modelos, a criação de deepfakes e a desinformação. Existe também a preocupação com o impacto nos empregos, especialmente em áreas de produção de conteúdo e design.',
  'meta': {'importância': 'alta'},
  'score': 0.97494996},
 {'id': 5,
  'text': 'Embora a IA generativa traga avanços significativos em personalização de conteúdo, como recomendações de produtos e serviços em plataformas de streaming, seu uso mais generalizado em marketing digital e publicidade também levanta questões sobre privacidade e ética no uso de dados pessoais.',
  'meta': {'

In [ ]:
rerank_request = RerankRequest(query=query, passages=passages)
output = ranker.rerank(rerank_request)

# Extrai o ranking gerado pelo modelo (baseado nas pontuações)
predicted_ranking = [result["id"] for result in sorted(output, key=lambda x: x["score"], reverse=True)]
predicted_ranking

In [ ]:
predicted_ranking

[1, 3, 5, 2, 4, 8, 7, 6]

## Comparação de rerank entre diferente modelos

In [29]:
from scipy.stats import kendalltau
import pandas as pd
import time
import numpy as np

In [9]:
# Modelos a serem usados
rankers = [
    {"name": "Ranker(max_length=128)", "instance": Ranker(max_length=128)},
    {"name": "Ranker(model_name='ms-marco-MiniLM-L-12-v2')", "instance": Ranker(model_name="ms-marco-MiniLM-L-12-v2")},
    {"name": "Ranker(model_name='rank-T5-flan')", "instance": Ranker(model_name="rank-T5-flan")},
    {"name": "Ranker(model_name='ms-marco-MultiBERT-L-12')", "instance": Ranker(model_name="ms-marco-MultiBERT-L-12")},
]

# Função para calcular as métricas
def calculate_metrics(true_ranking, predicted_ranking):
    # Kendall's Tau
    tau, _ = kendalltau(true_ranking, predicted_ranking)
    
    return tau



In [23]:
true_ranking = [result["id"] for result in sorted(passages, key=lambda x: x["id"], reverse=False)]
true_ranking

[1, 2, 3, 4, 5, 6, 7, 8]

In [30]:
# Processamento para cada modelo
results = []
for ranker in rankers:
    execution_times = []  # Lista para armazenar os tempos de execução para cada iteração
    
    # Executa o código 5 vezes
    for _ in range(5):
        start_time = time.time()
        # Realiza o reranking com o modelo
        rerank_request = RerankRequest(query=query, passages=passages)
        output = ranker["instance"].rerank(rerank_request)
        end_time = time.time()
        
        # Calcula o tempo de execução para a iteração
        execution_time = end_time - start_time
        execution_times.append(execution_time)
    
    # Calcula o tempo médio e o tempo mediano
    avg_execution_time = np.mean(execution_times)
    median_execution_time = np.median(execution_times)
    
    # Extrai o ranking gerado pelo modelo (baseado nas pontuações)
    predicted_ranking = [result["id"] for result in sorted(output, key=lambda x: x["score"], reverse=True)]
    
    # Calcula as métricas
    tau = calculate_metrics(true_ranking, predicted_ranking)
    
    # Armazena os resultados, incluindo o tempo médio e o tempo mediano
    results.append({
        "Modelo": ranker["name"],
        "Métrica": tau,
        "true_ranking": true_ranking,
        "predicted_ranking": predicted_ranking,
        "Tempo Médio (segundos)": avg_execution_time,
        "Tempo Mediano (segundos)": median_execution_time
    })


In [31]:
df = pd.DataFrame(results)

# Ordenando pelo valor da métrica (do maior ao menor)
df_sorted = df.sort_values(by='Métrica', ascending=False)

# Exibindo o DataFrame ordenado
print(df_sorted)

                                         Modelo   Métrica  \
3  Ranker(model_name='ms-marco-MultiBERT-L-12')  0.785714   
1  Ranker(model_name='ms-marco-MiniLM-L-12-v2')  0.714286   
0                        Ranker(max_length=128)  0.571429   
2             Ranker(model_name='rank-T5-flan') -0.428571   

               true_ranking         predicted_ranking  Tempo Médio (segundos)  \
3  [1, 2, 3, 4, 5, 6, 7, 8]  [2, 1, 3, 5, 4, 6, 8, 7]                0.563338   
1  [1, 2, 3, 4, 5, 6, 7, 8]  [3, 1, 2, 5, 4, 7, 6, 8]                0.217655   
0  [1, 2, 3, 4, 5, 6, 7, 8]  [1, 3, 5, 2, 4, 8, 7, 6]                0.015556   
2  [1, 2, 3, 4, 5, 6, 7, 8]  [7, 8, 4, 5, 2, 3, 1, 6]                0.808404   

   Tempo Mediano (segundos)  
3                  0.574229  
1                  0.217379  
0                  0.012233  
2                  0.815950  
